# Совпадение максимумов – случайность или происки шпионов?

#### Выполнил: Думбай А.Д., ВМК, гр. 517

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
from itertools import combinations
import plotly.plotly as py
import plotly.figure_factory as ff

sns.set()
%matplotlib inline

<img src="1.PNG">

Рассмотрим эту картинку и зададимся вопросом - в сбалансированном слуае всегда ли совпадают точки максимума?

Нас интересует две пары
1. roc_auc_score и accuracy_score
2. cohen_kappa_score и matthews_corrcoef

А так же взаимоотношения оценок из разных пар.

Для начала рассмотрим идеальный случай - когда размеры выборок с положительным классом и с отрицательным в точноси совпадают. Обозначим 
$n - \text{размер выборки}$.


### 1. roc_auc_score и accuracy_score

На семинаре было показано следующее выражение для AUC на бинаризованной выборке
$$\mathrm{AUC} =\frac { 1 } { 2 } \left( \frac { \mathrm { TN } } { \mathrm { FP } + \mathrm { TN } } + \frac { \mathrm { TP } } { \mathrm { TP } + \mathrm { FN } } \right)$$
В случае совпадающих размеров классов мы получим $ \mathrm { TP } + \mathrm { FN }  =\mathrm { FP } + \mathrm { TN }  = \frac{n}{2}$. Тогда формула переписывается следующим образом:
$$\mathrm{AUC} =\frac { 1 } { 2 } \left( \frac { 2\mathrm {TN } } {n } + \frac { 2\mathrm { TP } } { n} \right) = \frac{\mathrm{TP} + \mathrm{TN}}{n} = acc$$

В случае сбалансированных классов мы получаем полное совпадение данных оценок, что и проиллюстрировано на рисунке. 

* Данные функции для сбалансированных классов полностью совпадают, не только в максимуме. 
* Данные функции не зависят от распределений на классы, зависят только от того, совпадает ли число объектов положительного и отрицательного классов.

Исследуем, что будет происходить при разбалансировке. Положим $P = an, N = (1-a)n, n \in (0,1)$, тогда 

$$\mathrm{AUC} =\frac { 1 } { 2 } \left( \frac { \mathrm {TN } } {(1-a)n } + \frac { \mathrm { TP } } { an} \right) = = acc \frac {\frac{\mathrm {TN } } {(1-a) } + \frac { \mathrm { TP } } { a}}{2(\mathrm{TP} + \mathrm{TN})}$$

Как можно понять отсюда, точность корректируется с помощью некоторой суммы точно угаданных объектов первого  класса и второго и теперь зависит как от $a$, так и от вида распределения.

### 2. cohen_kappa_score

Рассмотрим cohen_kappa_score, обознаив его $r$. 

$$r = \frac { p _ { \text { observed } } - p _ { \text { chance } } } { 1 - p _ { \text { chance } } }$$

где $$ p _ { \text { observed }} = acc$$ 

$$p_{\text{chance}} = \frac {\mathrm {TP} + \mathrm {TN} } { n } \frac { \mathrm {TP }+ \mathrm {FN}} { n } + \frac { \mathrm {FN } + \mathrm {TN } } { n } \frac { \mathrm {FP} + \mathrm {TN } } { n } = \frac {\mathrm {TP} + \mathrm {FP} } { 2n } + \frac { \mathrm {FN } + \mathrm {TN } } { 2n }  = \frac{1}{2}$$

Получаем 
$$r = \frac{acc - \frac{1}{2}}{\frac{1}{2}} = 2acc - 1$$

В случае равенства размеров классов мы получаем прямую зависимость $r = 2 acc - 1$, что напрямую связывает эту функцию с предыдущими и, очевидно, равенство точек максимума достигается. 

Как видно из формулы при незначительном дисбалансе формула меняется так же не очень сильно, балансируется сам $p_{\text{chance}}$

### 3. matthews_corrcoef

$$\mathrm { MCC } = \frac { \mathrm { TP } \cdot \mathrm { TN } - \mathrm { FP } \cdot \mathrm { FN } } { \sqrt { ( \mathrm { TP } + \mathrm { FP } ) ( \mathrm { TP } + \mathrm { FN } ) ( \mathrm { TN } + \mathrm { FP } ) ( \mathrm { TN } + \mathrm { FN } ) } }$$

Воспользуемся равенством классов

$$\mathrm { MCC } = \frac { \mathrm { TP } \cdot \mathrm { TN } - \mathrm { FP } \cdot \mathrm { FN } } { \sqrt { ( \mathrm { TP } + \mathrm { FP } ) ( \frac{n}{2}) ( \frac{n}{2} ) ( \mathrm { TN } + \mathrm { FN } ) } } = \frac { n(\mathrm { TP } \cdot \mathrm { TN } - \mathrm { FP } \cdot \mathrm { FN }) } { 2\sqrt { ( \mathrm { TP } + \mathrm { FP } )  ( \mathrm { TN } + \mathrm { FN } ) } } $$


Тут уже прямого выражения не наблюдается. Будем рассуждать следующим образом, при изменении порога от большего к меньшему на каждом элементе происходит следующее - мы либо делаем правильно предсказанный негативный объект неправильно предсказанным положительным ($\mathrm { TN } \to \mathrm { FP }$), либо неправильно предсказанный негатиынфй - правильно предсказанным положительным($\mathrm { FN } \to \mathrm { TP } $), следовательно функция $f(h) = \mathrm { TN } + \mathrm {FN} $ - монотонно возрастающая функция равная $N$ - числу объектов, причисленных нами к отрицательному классу.

$$\mathrm { MCC } = \frac { 2(\mathrm { TP } \cdot \mathrm { TN } - \mathrm { FP } \cdot \mathrm { FN }) } { n\sqrt { N\cdot P} } = \frac { 2(\mathrm { TP } \cdot \mathrm { TN } - P\cdot N + P\cdot \mathrm { TN} + N\cdot\mathrm { TP } - \mathrm { TP }\cdot\mathrm { TN }) } { n\sqrt { NP} }  =  \frac { 2(P\cdot \mathrm { TN} + N\cdot \mathrm { TP } - P\cdot N ) } { n\sqrt { N\cdot P} }  = \frac{2}{n} \left(\mathrm { TN} \sqrt{\frac{P}{N}}  + \mathrm {TP} \sqrt{\frac{N}{P}} - \sqrt{P \cdot N}\right)$$

Эта функция выглядит так, в случае "симметричных" относительно некого своего центра, и одинаковых распределений на овтетах алгоритма кажется, что эта формула максимизируется где-то "посередине" между оветами, разбивая множество на две равные части. 

Но если распределения не семмитрины? Можно построить контрпримеры. Видно, то сами TP и TN "балансирутся" дисбалансом предсказанных классов в некотором роде, но при этом этого может быть недостаточно.

Сгенерируем __неправильное__ распределение, где на концах будут чистые классы, а в середине - равномерная смесь. 

Пусть у нас ответы алгоритма идут по равномерной сетке от -1 до 1. Первые 100 имеют метку -1, вторые 100 - чередуясь 1 и -1, последние 100 - метку 1.

In [2]:
from sklearn.metrics import matthews_corrcoef

In [3]:
y = np.hstack((-np.ones((100)), [-1, 1] * 100, np.ones((100))))
h = list(range(1, y.shape[0] - 1))
y_2 = -np.ones(y.shape)
y_2[-1] = 1
scores = []
for hh in range(y.shape[0] - 1, 1, -1):
    y_2[hh] = 1
    scores.append(matthews_corrcoef(y, y_2))

In [4]:
random_x = np.linspace(-1, 1, len((scores)))
random_y = scores

trace = go.Scatter(
    x = random_x,
    y = random_y
)

layout = dict(title = 'MCC в зависимости от порога',
               xaxis= dict(
                    title= 'Порог',
                    ticklen= 5,
                    zeroline= False,
                    gridwidth= 1,
                    dtick=1,
                ),
                yaxis=dict(
                    title= 'matthews_corrcoef',
                    ticklen= 5,
                    gridwidth= 1,
                ),
             )

plot_data = [trace]

py.iplot({'data': plot_data, 'layout':layout}, filename='basic-line')

Коэффициент, как и ожидалось, симметричен, но максимум достигается отнюдь не в центре, а по краям, за счет более чистого предсказывания классов. При малом числе серединных-смешанных элементов центральная часть скорее вырождается в плато или в точку максимума.

Еще раз проиллюстрируем пример уже на случайной ситуации, где по краям будет столько же элемнтов, что и в середине

Пусть положительный класс приходит из равномерного на [-1,1] распеделения(1000 элементов) по ответам алгоритмов и из нормального $N(2, 1)$(1000 элементов) по ответам, отрицательный класс - из равномерного на [0,1](1000 элементов) по ответам и $N(-2, 1)$(1000 элементов) по ответам. Получаем x - ответ алгоритма.


In [5]:
x_p = np.random.randn(1000) + 2
x_n = np.random.randn(1000) - 2
x_p_u = np.random.rand(1000) * 2 - 1
x_n_u = np.random.rand(1000) * 2 - 1

In [6]:
x = np.hstack((x_p, x_p_u, x_n_u, x_n))
y = np.ones((x.shape[0]//2))
y = np.hstack((y, -y))

Посмотрим на распределения.

In [7]:
hist_data = [np.hstack((x_p, x_p_u)), np.hstack((x_n, x_n_u))]


group_labels = ['Positive', 'Negative']

fig = ff.create_distplot(hist_data, group_labels, bin_size=.2)

py.iplot(fig, filename='Распределение')

In [8]:
h = np.sort(x)
scores = []
for hh in h[1:-1]:
    # перебираем тупо, в отличии от предыдущего эксперимента, потому что можем
    y_2 = y.copy()
    y_2[x >= hh] = 1
    y_2[x < hh] = -1
    scores.append(matthews_corrcoef(y, y_2))

In [9]:
trace = go.Scatter(
    x = h,
    y = scores
)

layout = dict(title = 'MCC в зависимости от порога',
               xaxis= dict(
                    title= 'Порог',
                    ticklen= 5,
                    zeroline= False,
                    gridwidth= 1,
                    dtick=1,
                ),
                yaxis=dict(
                    title= 'matthews_corrcoef',
                    ticklen= 5,
                    gridwidth= 1,
                ),
             )

plot_data = [trace]

py.iplot({'data': plot_data, 'layout':layout}, filename='real_line')

Результат такой же, как и ожидалось. Средняя часть еще меньше, чем в предыдущем эксперименте, максимумы на концах. При этом падение в "среденей" область достаточно незначительно, по сравнению с максимумом.

Следовательно MCC с остальными в точке максимума на картинке совпадает случайно, что вызвано струткурой распределений на ответах, а не балансом выборки.

## Выводы

ROC_AUC и Accuracy для сбалансированных классов - суть одно и то же, cohen_kappa_score - их линейное преобразование. Что показано в теоретиеских выкладках. При небольшом отклонении от идеального случая эти равенства сильно не меняются, но это зависит и от распределений на ответах.

В слуае же с MCC наблюдаемая на исходной картинке закономерность - лишь совпадение, вызванное симметрией относительно медианы нормального распределения, что было показано в эксперименте. В нем было приведено множество, образующего "тяжелые крылья" с равномерной средней частью, в результате чего, даже при равенстве мощностей классов образовалось два максимума, удаленные от центра. Это вызвано тем, что MCC в сравнении с другими тремя метриками зависит не только от баланса, но и от схожести распределений и их симметричности относительно "центра".